```
"reply_id" = le numéro dans npc.###.reply
"message_id" = le numéro dans npc.###.message
 
"reply_parent_id" = npc.###.message auquel est lié la réponse
"reply_message_id" = npc.###.message qui s'affichera si on clique sur la réponse

"dialog_id" = c'est sa clé unique pour nous pouvoir le modifier via requête
"dialog_npc_id" = c'est la clé du pnj dans le fichier npc
"dialog_message_id" = c'est la clé du texte à afficher dans le ficher message
"dialog_message_check_order" = c'est dans le cas où plusieurs dialogues sont possible sur le pnj, ils sont filtrées en fonction de condition (quête, map, etc...) et si plusieurs dialogues sont valides on affiche le premier en fonction du check_order
```


# Paths to folder and source dialog files

In [2]:
FOLDER = "Retro_json_dialog_data"

PATH_DIALOG = "export_npc_dialog_json.json"
PATH_NPC = "export_npc_json.json"
PATH_MESSAGE = "export_npc_message_json.json"
PATH_REPLY = "export_npc_reply_json.json"
PATH_METADATA = "npcs_Dofus3-03_202508.json"

# SRC - Dialog Mapping and HTML rendering functions

In [9]:
import json
import os
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, field
from collections import defaultdict

@dataclass
class NPCMessage:
    message_id: int
    text_fr: str
    text_en: Optional[str]
    text_es: Optional[str]
    text_pt: Optional[str]
    context: Optional[str]
    replies: List['NPCReply'] = field(default_factory=list)

@dataclass
class NPCReply:
    reply_id: int
    text_fr: str
    text_en: Optional[str]
    text_es: Optional[str]
    text_pt: Optional[str]
    context: Optional[str]
    parent_message_id: int
    leads_to_message_id: Optional[int]

@dataclass
class NPCDialog:
    dialog_id: int
    npc_id: int
    message_id: int
    check_order: int

@dataclass
class NPCMetadata:
    index_id: int
    gender: int  # 0 = MALE, 1 = FEMALE, 2 = UNDEFINED
    look: str
    img_url: str
    metadata_id: str
    name_en: str
    name_pt: str
    name_es: str
    name_fr: str
    name_de: str

@dataclass
class NPC:
    npc_id: int
    name_fr: str
    name_en: Optional[str]
    name_es: Optional[str]
    name_pt: Optional[str]
    context: Optional[str]
    dialogs: List[NPCDialog] = field(default_factory=list)
    # Enhanced with metadata from sister game
    genders: List[int] = field(default_factory=list)  # Can have multiple genders if multiple matches
    img_urls: List[str] = field(default_factory=list)  # Can have multiple images if multiple matches

class NPCDialogMapper:
    def __init__(self, folder_path: str):
        self.folder_path = folder_path
        self.npcs: Dict[int, NPC] = {}
        self.messages: Dict[int, NPCMessage] = {}
        self.replies: Dict[int, NPCReply] = {}
        self.dialogs: List[NPCDialog] = []
        self.metadata: List[NPCMetadata] = []
        
    def load_data(self):
        """Load all JSON data files"""
        print("Loading NPC metadata...")
        self._load_metadata()
        print("Loading NPC data...")
        self._load_npcs()
        print("Loading messages...")
        self._load_messages()
        print("Loading replies...")
        self._load_replies()
        print("Loading dialogs...")
        self._load_dialogs()
        print("Building relationships...")
        self._build_relationships()
        print("Matching metadata with NPCs...")
        self._match_metadata_with_npcs()
        
    def _load_metadata(self):
        """Load NPC metadata from sister game"""
        file_path = os.path.join(self.folder_path, PATH_METADATA)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        for item in data:
            self.metadata.append(NPCMetadata(
                index_id=item['indexID'],
                gender=item['gender'],
                look=item['look'],
                img_url=item['img'],
                metadata_id=item['id'],
                name_en=item['en'],
                name_pt=item['pt'],
                name_es=item['es'],
                name_fr=item['fr'],
                name_de=item['de']
            ))
        
    def _load_npcs(self):
        """Load NPC data"""
        file_path = os.path.join(self.folder_path, PATH_NPC)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        # Get the first (and only) key which contains the SQL query
        npc_data = list(data.values())[0]
        
        for npc in npc_data:
            self.npcs[npc['npc_id']] = NPC(
                npc_id=npc['npc_id'],
                name_fr=npc['npc_name_fr'],
                name_en=npc['npc_name_en'],
                name_es=npc['npc_name_es'],
                name_pt=npc['npc_name_pt'],
                context=npc['context']
            )
    
    def _load_messages(self):
        """Load message data"""
        file_path = os.path.join(self.folder_path, PATH_MESSAGE)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        message_data = list(data.values())[0]
        
        for msg in message_data:
            self.messages[msg['message_id']] = NPCMessage(
                message_id=msg['message_id'],
                text_fr=msg['message_text_fr'],
                text_en=msg['message_text_en'],
                text_es=msg['message_text_es'],
                text_pt=msg['message_text_pt'],
                context=msg['message_i18n_context']
            )
    
    def _load_replies(self):
        """Load reply data"""
        file_path = os.path.join(self.folder_path, PATH_REPLY)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        reply_data = list(data.values())[0]
        
        for reply in reply_data:
            self.replies[reply['reply_id']] = NPCReply(
                reply_id=reply['reply_id'],
                text_fr=reply['reply_text_fr'],
                text_en=reply['reply_text_en'],
                text_es=reply['reply_text_es'],
                text_pt=reply['reply_text_pt'],
                context=reply['reply_i18n_context'],
                parent_message_id=reply['reply_parent_id'],
                leads_to_message_id=reply['reply_message_id']
            )
    
    def _load_dialogs(self):
        """Load dialog data"""
        file_path = os.path.join(self.folder_path, PATH_DIALOG)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        dialog_data = list(data.values())[0]
        
        for dialog in dialog_data:
            dialog_obj = NPCDialog(
                dialog_id=dialog['dialog_id'],
                npc_id=dialog['dialog_npc_id'],
                message_id=dialog['dialog_message_id'],
                check_order=dialog['dialog_message_check_order']
            )
            self.dialogs.append(dialog_obj)
    
    def _build_relationships(self):
        """Build relationships between NPCs, dialogs, messages and replies"""
        # Link dialogs to NPCs
        for dialog in self.dialogs:
            if dialog.npc_id in self.npcs:
                self.npcs[dialog.npc_id].dialogs.append(dialog)
        
        # Sort dialogs by check_order for each NPC
        for npc in self.npcs.values():
            npc.dialogs.sort(key=lambda d: d.check_order)
        
        # Link replies to messages
        for reply in self.replies.values():
            if reply.parent_message_id in self.messages:
                self.messages[reply.parent_message_id].replies.append(reply)
    
    def _match_metadata_with_npcs(self):
        """Match metadata with NPCs by French name"""
        metadata_by_name = defaultdict(list)
        
        # Group metadata by French name
        for meta in self.metadata:
            metadata_by_name[meta.name_fr].append(meta)
        
        # Match NPCs with metadata
        matched_count = 0
        for npc in self.npcs.values():
            if npc.name_fr in metadata_by_name:
                matches = metadata_by_name[npc.name_fr]
                
                # Extract unique genders and image URLs
                unique_genders = list(set(meta.gender for meta in matches))
                unique_img_urls = list(set(meta.img_url for meta in matches))
                
                npc.genders = unique_genders
                npc.img_urls = unique_img_urls
                matched_count += 1
        
        print(f"Matched {matched_count} NPCs with metadata from {len(self.npcs)} total NPCs")
    
    def generate_html(self, output_file: str = "npc_dialog_mapping.html"):
        """Generate responsive HTML file with all mapped data"""
        print(f"Generating HTML file: {output_file}")
        
        html_content = self._generate_html_content()
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"HTML file generated successfully: {output_file}")
    
    def _format_text_with_null_check(self, text: Optional[str], lang: str = "") -> str:
        """Format text with null checking and warning for null values"""
        if text is None or text == "":
            return f'<span class="null-value">NULL ⚠️</span>'
        return text.replace('\n', '<br>').replace('\r', '')
    
    def _remove_diacritics(self, text: str) -> str:
        """Remove diacritics from text for search comparison"""
        import unicodedata
        return ''.join(c for c in unicodedata.normalize('NFD', text) 
                      if unicodedata.category(c) != 'Mn')
    
    def _wildcard_to_regex(self, pattern: str) -> str:
        """Convert wildcard pattern to regex"""
        import re
        # Escape special regex characters except *
        escaped = re.escape(pattern)
        # Replace escaped asterisks with regex pattern
        regex_pattern = escaped.replace(r'\*', r'[^\s]*')
        return regex_pattern
    
    def _highlight_matches(self, text: str, search_term: str, ignore_diacritics: bool = False, use_wildcards: bool = False) -> str:
        """Highlight search matches in text"""
        if not search_term or not text:
            return text
            
        import re
        
        # Prepare search term and text for comparison
        search_text = text
        pattern = search_term
        
        if ignore_diacritics:
            search_text = self._remove_diacritics(text)
            pattern = self._remove_diacritics(search_term)
        
        if use_wildcards and '*' in pattern:
            pattern = self._wildcard_to_regex(pattern)
            flags = re.IGNORECASE
        else:
            pattern = re.escape(pattern)
            flags = re.IGNORECASE
        
        try:
            # Find matches in the processed text
            matches = list(re.finditer(pattern, search_text, flags))
            
            if not matches:
                return text
            
            # Apply highlighting to original text
            result = text
            offset = 0
            
            for match in matches:
                start, end = match.span()
                # Adjust positions for original text
                start += offset
                end += offset
                
                original_match = result[start:end]
                highlighted = f'<span class="search-highlight">{original_match}</span>'
                result = result[:start] + highlighted + result[end:]
                
                # Update offset for next replacement
                offset += len('<span class="search-highlight">') + len('</span>')
            
            return result
        except re.error:
            # If regex fails, return original text
            return text
    
    def _generate_html_content(self) -> str:
        """Generate the complete HTML content"""
        
        # Generate NPC cards HTML
        npc_cards_html = ""
        
        for npc in sorted(self.npcs.values(), key=lambda x: x.name_fr.lower()):
            if not npc.dialogs:  # Skip NPCs without dialogs
                continue
                
            npc_card = self._generate_npc_card(npc)
            npc_cards_html += npc_card
        
        # Complete HTML template
        html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>NPC Dialog Mapping - Dofus Retro</title>
    <style>
        {self._get_css_styles()}
    </style>
</head>
<body>
    <header>
        <h1>🗣️ NPC Dialog Mapping - Dofus Retro</h1>
        <button id="menuToggle" class="menu-toggle" title="Help & Guide">☰</button>
        <div class="disclaimer">
            ⚠️ <strong>Disclaimer:</strong> Gender and images are retrieved from name matches with Dofus 3 (dofusdb fansite), they might differ from RETRO's NPCs.
        </div>
        <div class="search-container">
            <div class="search-box">
                <input type="text" id="searchInput" placeholder="Search NPCs, dialogs, or IDs... (use * for wildcards)">
                
                <div class="search-options-grid">
                    <!-- Search Type Options -->
                    <div class="option-group">
                        <h4>🔍 Search Type</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="radio" name="searchType" value="text" checked><span class="option-text">📝 Search in Text</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="npc-name"><span class="option-text">🧙‍♂️ Search in NPC Name</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="npc-id"><span class="option-text">🆔 Search by NPC ID</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="message-id"><span class="option-text">💬 Search by Message ID</span></label>
                            <label class="option-row"><input type="radio" name="searchType" value="reply-id"><span class="option-text">↪️ Search by Reply ID</span></label>
                        </div>
                    </div>
                    
                    <!-- Text Search Options -->
                    <div class="option-group">
                        <h4>⚙️ Text Search Options</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" id="exactMatch"><span class="option-text">🎯 Exact match</span></label>
                            <label class="option-row"><input type="checkbox" id="ignoreDiacritics"><span class="option-text">📝 Ignore diacritics (é=e)</span></label>
                            <label class="option-row"><input type="checkbox" id="useWildcards"><span class="option-text">⭐ Use wildcards (*)</span></label>
                            <label class="option-row"><input type="checkbox" id="showOnlyMatchDialogs"><span class="option-text">🌳 Show only match's dialog tree</span></label>
                            <label class="option-row"><input type="checkbox" id="showOnlyMatchMsgReply"><span class="option-text">💬↪️ Show only match's msg&reply</span></label>
                        </div>
                    </div>
                    
                    <!-- Language Filters -->
                    <div class="option-group">
                        <h4>🔎 Search in Languages</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" name="searchLang" value="fr" checked><span class="option-text">FR</span></label>
                            <label class="option-row"><input type="checkbox" name="searchLang" value="en" checked><span class="option-text">EN</span></label>
                            <label class="option-row"><input type="checkbox" name="searchLang" value="es" checked><span class="option-text">ES</span></label>
                            <label class="option-row"><input type="checkbox" name="searchLang" value="pt" checked><span class="option-text">PT</span></label>
                        </div>
                    </div>
                    
                    <!-- Display Options -->
                    <div class="option-group">
                        <h4>🖥️ Display Languages</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" name="displayLang" value="fr" checked><span class="option-text">Show FR</span></label>
                            <label class="option-row"><input type="checkbox" name="displayLang" value="en" checked><span class="option-text">Show EN</span></label>
                            <label class="option-row"><input type="checkbox" name="displayLang" value="es" checked><span class="option-text">Show SP</span></label>
                            <label class="option-row"><input type="checkbox" name="displayLang" value="pt" checked><span class="option-text">Show PT</span></label>
                        </div>
                    </div>
                    
                    <!-- Other Options -->
                    <div class="option-group">
                        <h4>🔧 Other Options</h4>
                        <div class="option-table">
                            <label class="option-row"><input type="checkbox" id="showNullOnly"><span class="option-text">⚠️ Show NULL values only</span></label>
                            <label class="option-row"><input type="checkbox" id="enableImages"><span class="option-text">🖼️ Enable NPC images on hover</span></label>
                            <label class="option-row"><input type="checkbox" id="enableConsolas"><span class="option-text">📝 Use Consolas font</span></label>
                            <label class="option-row language-highlight-row">
                                <span class="option-text">🌟 Highlight my language:</span>
                                <select id="highlightLanguage" class="language-dropdown">
                                    <option value="">None</option>
                                    <option value="fr">FR</option>
                                    <option value="en">EN</option>
                                    <option value="es">ES</option>
                                    <option value="pt">PT</option>
                                </select>
                            </label>
                        </div>
                    </div>
                </div>
                
                <div class="search-controls">
                    <button id="applyFiltersBtn" class="apply-btn">
                        <span class="btn-text">Apply Search Filters</span>
                        <span class="btn-icon" style="display: none;">🔄</span>
                    </button>
                    <button id="collapseAllBtn" class="control-btn">Collapse All</button>
                    <button id="expandAllBtn" class="control-btn">Expand All</button>
                </div>
            </div>
        </div>
        <div class="stats">
            <span>Total NPCs: {len([npc for npc in self.npcs.values() if npc.dialogs])}</span>
            <span>Total Messages: {len(self.messages)}</span>
            <span>Total Replies: {len(self.replies)}</span>
        </div>
    </header>
    
    <main id="npcContainer">
        {npc_cards_html}
    </main>
    
    <!-- Floating Navigation -->
    <div id="floatingNav" style="display: none;">
        <button id="prevMatchBtn" class="nav-btn" title="Previous Match">↑</button>
        <span id="matchCounter">0/0</span>
        <button id="nextMatchBtn" class="nav-btn" title="Next Match">↓</button>
    </div>
    
    <!-- Side Menu -->
    <div id="sideMenu" class="side-menu">
        <div class="side-menu-header">
            <h2>❓ Help & User Guide</h2>
            <button id="closeSideMenu" class="close-menu-btn">✕</button>
        </div>
        <div class="side-menu-content">
            <div class="help-section">
                <h3>🔍 Search Types</h3>
                <ul>
                    <li><strong>📝 Search in Text:</strong> Search within message and reply content in all languages</li>
                    <li><strong>🧙‍♂️ Search in NPC Name:</strong> Search specifically in NPC names</li>
                    <li><strong>🆔 Search by NPC ID:</strong> Find NPCs by their unique ID number</li>
                    <li><strong>💬 Search by Message ID:</strong> Find specific messages by ID</li>
                    <li><strong>↪️ Search by Reply ID:</strong> Find specific replies by ID</li>
                </ul>
            </div>
            
            <div class="help-section">
                <h3>⚙️ Text Search Options</h3>
                <ul>
                    <li><strong>🎯 Exact match:</strong> Search for whole words only (e.g., "cat" won't match "category")</li>
                    <li><strong>📝 Ignore diacritics:</strong> "café" will match "cafe", "résumé" will match "resume"</li>
                    <li><strong>⭐ Use wildcards:</strong> Use * as wildcard (e.g., "hel*" matches "hello", "help", "helmet")</li>
                    <li><strong>🌳 Show only match's dialog tree:</strong> Show complete dialog trees that contain matches</li>
                    <li><strong>💬↪️ Show only match's msg&reply:</strong> Show only matching messages with their direct replies, or matching replies with their parent messages</li>
                </ul>
            </div>
            
            <div class="help-section">
                <h3>🌐 Language Options</h3>
                <ul>
                    <li><strong>🔎 Search in Languages:</strong> Choose which languages to search in</li>
                    <li><strong>🖥️ Display Languages:</strong> Choose which languages to show in results</li>
                    <li><strong>🌟 Highlight my language:</strong> Highlight your preferred language with a dotted border</li>
                </ul>
            </div>
            
            <div class="help-section">
                <h3>🔧 Other Features</h3>
                <ul>
                    <li><strong>⚠️ Show NULL values only:</strong> Display only entries with missing translations</li>
                    <li><strong>🖼️ Enable NPC images:</strong> Show character images when hovering over NPC cards</li>
                    <li><strong>Collapse/Expand All:</strong> Quickly hide or show all dialog content</li>
                    <li><strong>Navigation:</strong> Use the floating navigation to jump between search results</li>
                </ul>
            </div>
            
            <div class="help-section">
                <h3>💡 Tips & Examples</h3>
                <ul>
                    <li><strong>Wildcard search:</strong> "quest*" finds "quest", "question", "questionnaire"</li>
                    <li><strong>Exact match:</strong> "the" only matches whole word "the", not "them" or "theory"</li>
                    <li><strong>Diacritic search:</strong> "gele" will find "gelé" & "gèle" when enabled</li>
                    <li><strong>Multi-language search:</strong> Uncheck languages you don't want to search in for faster results</li>
                    <li><strong>NULL hunting:</strong> Use "Show NULL values only" to find missing translations</li>
                </ul>
            </div>
        </div>
    </div>
    
    <!-- Side menu overlay -->
    <div class="side-menu-overlay" id="sideMenuOverlay"></div>
    
    <script>
        {self._get_javascript()}
    </script>
</body>
</html>
        """
        
        return html_template.strip()
    
    def _generate_npc_card(self, npc: NPC) -> str:
        """Generate HTML card for a single NPC"""
        
        # Format gender information with pastel colors
        gender_info = ""
        if npc.genders:
            gender_items = []
            for gender in npc.genders:
                if gender == 0:
                    gender_items.append('<span class="gender-male">Male</span>')
                elif gender == 1:
                    gender_items.append('<span class="gender-female">Female</span>')
                elif gender == 2:
                    gender_items.append('<span class="gender-undefined">Undefined</span>')
            gender_info = f'<div class="npc-gender"><strong>Gender:</strong> {", ".join(gender_items)}</div>'
        
        # Format image URLs for hover display
        img_data_attr = ""
        if npc.img_urls:
            img_data_attr = f'data-img-urls="{"|".join(npc.img_urls)}"'
        
        dialogs_html = ""
        for dialog in npc.dialogs:
            if dialog.message_id in self.messages:
                message = self.messages[dialog.message_id]
                dialog_html = self._generate_dialog_tree(message, dialog, level=0)
                dialogs_html += f"""
                <div class="dialog-container collapsible">
                    <div class="dialog-header clickable-header">
                        <strong>Dialog ID: {dialog.dialog_id}</strong> 
                        (Check Order: {dialog.check_order})
                        <span class="collapse-indicator">−</span>
                    </div>
                    <div class="collapsible-content">
                        {dialog_html}
                    </div>
                </div>
                """
        
        return f"""
        <div class="npc-card collapsible" data-npc-id="{npc.npc_id}" {img_data_attr}>
            <div class="npc-header sticky-header">
                <div class="header-content">
                    <h2 class="clickable-header">🧙‍♂️ {npc.name_fr} <span class="npc-id">(ID: {npc.npc_id})</span> <span class="collapse-indicator">−</span></h2>
                    <div class="npc-info-grid collapsible-content">
                        <div class="npc-names">
                            <div class="lang-row">
                                <span class="lang-label">FR:</span> {self._format_text_with_null_check(npc.name_fr)}
                            </div>
                            <div class="lang-row">
                                <span class="lang-label">EN:</span> {self._format_text_with_null_check(npc.name_en)}
                            </div>
                            <div class="lang-row">
                                <span class="lang-label">ES:</span> {self._format_text_with_null_check(npc.name_es)}
                            </div>
                            <div class="lang-row">
                                <span class="lang-label">PT:</span> {self._format_text_with_null_check(npc.name_pt)}
                            </div>
                        </div>
                        <div class="npc-metadata">
                            {gender_info}
                            {f'<div class="npc-context"><strong>Context:</strong> {npc.context}</div>' if npc.context else ''}
                        </div>
                    </div>
                </div>
                <div class="npc-image-container" style="display: none;">
                    <img class="npc-image" src="" alt="NPC Image" />
                </div>
            </div>
            <div class="dialogs collapsible-content">
                {dialogs_html}
            </div>
        </div>
        """
    
    def _generate_dialog_tree(self, message: NPCMessage, dialog: NPCDialog, level: int = 0, visited: set = None) -> str:
        """Generate HTML for dialog tree with message-reply hierarchy"""
        if visited is None:
            visited = set()
        
        # Prevent infinite loops
        if message.message_id in visited:
            return f'<div class="message-ref">↺ Reference to Message ID: {message.message_id}</div>'
        
        visited.add(message.message_id)
        
        # Cap the indentation at a reasonable level to prevent overflow
        max_level = 15  # Maximum visual indentation level
        visual_level = min(level, max_level)
        indent_style = f"margin-left: {visual_level * 10}px;"  # Reduced from 20px to 10px
        
        # Add depth indicator for very deep levels
        depth_indicator = f" (Depth: {level})" if level > max_level else ""
        
        message_html = f"""
        <div class="message collapsible depth-{visual_level}" data-message-id="{message.message_id}" data-depth="{level}">
            <div class="message-header clickable-header">
                {f'<span class="depth-badge">Depth: {level}</span>' if level > max_level else ''}
                <strong>💬 Message ID: {message.message_id}</strong>
                <span class="collapse-indicator">−</span>
            </div>
            <div class="message-content collapsible-content">
                <div class="lang-row">
                    <span class="lang-label">FR:</span> {self._format_text_with_null_check(message.text_fr)}
                </div>
                <div class="lang-row">
                    <span class="lang-label">EN:</span> {self._format_text_with_null_check(message.text_en)}
                </div>
                <div class="lang-row">
                    <span class="lang-label">ES:</span> {self._format_text_with_null_check(message.text_es)}
                </div>
                <div class="lang-row">
                    <span class="lang-label">PT:</span> {self._format_text_with_null_check(message.text_pt)}
                </div>
            </div>
        """
        
        # Add replies within the collapsible content so they get hidden when message is collapsed
        if message.replies:
            message_html += '<div class="replies collapsible-nested">'
            for reply in message.replies:
                # Cap reply visual level as well
                reply_visual_level = min(level + 1, max_level)
                reply_indent_style = f"margin-left: {reply_visual_level * 10}px;"  # Reduced from 20px to 10px
                reply_depth_indicator = f" (Depth: {level + 1})" if level + 1 > max_level else ""
                
                reply_html = f"""
                <div class="reply collapsible-nested depth-{reply_visual_level}" data-reply-id="{reply.reply_id}" data-depth="{level + 1}">
                    <div class="reply-header">
                        {f'<span class="depth-badge">Depth: {level + 1}</span>' if level + 1 > max_level else ''}
                        <strong>↪️ Reply ID: {reply.reply_id}</strong>
                        {f' → Message ID: {reply.leads_to_message_id}' if reply.leads_to_message_id else ''}
                    </div>
                    <div class="reply-content">
                        <div class="lang-row">
                            <span class="lang-label">FR:</span> {self._format_text_with_null_check(reply.text_fr)}
                        </div>
                        <div class="lang-row">
                            <span class="lang-label">EN:</span> {self._format_text_with_null_check(reply.text_en)}
                        </div>
                        <div class="lang-row">
                            <span class="lang-label">ES:</span> {self._format_text_with_null_check(reply.text_es)}
                        </div>
                        <div class="lang-row">
                            <span class="lang-label">PT:</span> {self._format_text_with_null_check(reply.text_pt)}
                        </div>
                    </div>
                """
                
                # If reply leads to another message, recursively add it within the reply
                if reply.leads_to_message_id and reply.leads_to_message_id in self.messages:
                    next_message = self.messages[reply.leads_to_message_id]
                    reply_html += '<div class="nested-messages">'
                    reply_html += self._generate_dialog_tree(next_message, dialog, level + 2, visited.copy())
                    reply_html += '</div>'
                
                reply_html += '</div>'
                message_html += reply_html
            
            message_html += '</div>'
        
        message_html += '</div>'
        visited.remove(message.message_id)
        
        return message_html
    
    def _get_css_styles(self) -> str:
        """Get CSS styles for the HTML"""
        return """
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        
        /* Consolas font override */
        body.consolas-font {
            font-family: 'Consolas', 'Courier New', monospace;
        }
        
        body.consolas-font * {
            font-family: 'Consolas', 'Courier New', monospace;
        }
        
        header {
            background: white;
            padding: 20px;
            border-radius: 15px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        
        h1 {
            color: #333;
            margin-bottom: 20px;
            text-align: center;
        }
        
        .disclaimer {
            background: #fff3cd;
            border: 1px solid #ffeaa7;
            border-radius: 8px;
            padding: 12px;
            margin-bottom: 20px;
            text-align: center;
            color: #856404;
        }
        
        .search-container {
            max-width: 800px;
            margin: 0 auto 20px;
        }
        
        .search-box input {
            width: 100%;
            padding: 12px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 8px;
            margin-bottom: 15px;
        }
        
        .search-options-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin-bottom: 15px;
        }
        
        .option-group {
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            border: 1px solid #e9ecef;
        }
        
        .option-group h4 {
            margin: 0 0 15px 0;
            color: #495057;
            font-size: 14px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }
        
        /* Table-like layout for options */
        .option-table {
            display: flex;
            flex-direction: column;
            gap: 8px;
        }
        
        .option-row {
            display: flex;
            align-items: flex-start;
            padding: 8px 12px;
            border-radius: 4px;
            transition: background-color 0.2s ease;
            cursor: pointer;
            margin: 0;
            min-height: 32px;
            gap: 8px;
        }
        
        .option-row:hover {
            background-color: rgba(76, 175, 80, 0.1);
        }
        
        .option-row input[type="radio"],
        .option-row input[type="checkbox"] {
            margin: 0;
            flex-shrink: 0;
            width: 16px;
            height: 16px;
            margin-top: 2px;
        }
        
        .option-text {
            flex: 1;
            font-size: 14px;
            line-height: 1.4;
            word-wrap: break-word;
            overflow-wrap: break-word;
            max-width: calc(100% - 32px);
        }
        
        .language-highlight-row {
            justify-content: space-between;
            align-items: center;
        }
        
        .language-highlight-row .option-text {
            flex: 0 0 auto;
            margin-right: 10px;
        }
        
        .search-controls {
            text-align: center;
            margin-top: 15px;
            display: flex;
            gap: 10px;
            justify-content: center;
            flex-wrap: wrap;
        }
        
        .apply-btn, .control-btn {
            background: linear-gradient(135deg, #4CAF50, #45a049);
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 8px;
            font-size: 16px;
            cursor: pointer;
            transition: all 0.3s ease;
            display: inline-flex;
            align-items: center;
            gap: 8px;
        }
        
        .control-btn {
            background: linear-gradient(135deg, #2196F3, #1976D2);
            padding: 10px 16px;
            font-size: 14px;
        }
        
        .apply-btn:hover, .control-btn:hover {
            transform: translateY(-1px);
        }
        
        .apply-btn.changed {
            background: linear-gradient(135deg, #ff9800, #f57c00);
        }
        
        .apply-btn.changed .btn-icon {
            display: inline !important;
        }
        
        .stats {
            text-align: center;
            color: #666;
            font-size: 14px;
        }
        
        .stats span {
            margin: 0 15px;
        }
        
        .npc-card {
            background: white;
            border-radius: 15px;
            margin-bottom: 20px;
            overflow: hidden;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: transform 0.2s;
        }
        
        .npc-card:hover {
            transform: translateY(-2px);
        }
        
        .npc-header {
            background: linear-gradient(135deg, #ff6b6b, #ffa726);
            color: white;
            padding: 20px;
        }
        
        .npc-header h2 {
            margin-bottom: 15px;
        }
        
        .npc-id {
            font-size: 0.8em;
            opacity: 0.9;
        }
        
        .npc-names {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 10px;
            margin-bottom: 10px;
        }
        
        .npc-info-grid {
            display: grid;
            grid-template-columns: 2fr 1fr;
            gap: 20px;
            margin-bottom: 10px;
        }
        
        .npc-metadata {
            display: flex;
            flex-direction: column;
            gap: 10px;
        }
        
        .npc-gender {
            background: rgba(255,255,255,0.2);
            padding: 8px;
            border-radius: 6px;
            font-size: 0.9em;
        }
        
        .gender-male {
            background: #b3d9ff;
            color: #0066cc;
            padding: 2px 8px;
            border-radius: 12px;
            font-weight: bold;
            font-size: 0.85em;
        }
        
        .gender-female {
            background: #ffb3d9;
            color: #cc0066;
            padding: 2px 8px;
            border-radius: 12px;
            font-weight: bold;
            font-size: 0.85em;
        }
        
        .gender-undefined {
            background: #e6e6e6;
            color: #666666;
            padding: 2px 8px;
            border-radius: 12px;
            font-weight: bold;
            font-size: 0.85em;
        }
        
        .search-highlight {
            background-color: #ffff99;
            padding: 1px 2px;
            border-radius: 2px;
        }
        
        .current-match {
            outline: 3px solid #ff6b6b;
            outline-offset: 2px;
            border-radius: 8px;
        }
        
        .npc-image-container {
            position: absolute;
            top: 10px;
            right: 10px;
            z-index: 10;
            background: rgba(255,255,255,0.9);
            border-radius: 8px;
            padding: 5px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.3);
        }
        
        .npc-image {
            width: 150px;
            height: 150px;
            border-radius: 4px;
        }
        
        .npc-header {
            position: relative;
        }
        
        .sticky-header {
            position: sticky;
            top: 0;
            z-index: 100;
            background: linear-gradient(135deg, #ff6b6b, #ffa726);
            border-radius: 15px 15px 0 0;
        }
        
        .clickable-header {
            cursor: pointer;
            user-select: none;
            transition: opacity 0.2s ease;
        }
        
        .clickable-header:hover {
            opacity: 0.8;
        }
        
        .collapse-indicator {
            float: right;
            font-weight: bold;
            transition: transform 0.3s ease;
        }
        
        .collapsible.collapsed .collapse-indicator {
            transform: rotate(-90deg);
        }
        
        .collapsible.collapsed .collapsible-content {
            display: none !important;
        }
        
        .collapsible.collapsed .collapsible-nested {
            display: none !important;
        }
        
        .collapsible.collapsed .nested-messages {
            display: none !important;
        }
        
        .collapsible-content {
            transition: all 0.3s ease;
        }
        
        .collapsible-nested {
            transition: all 0.3s ease;
        }
        
        .nested-messages {
            transition: all 0.3s ease;
        }
        
        #floatingNav {
            position: fixed;
            top: 50%;
            right: 20px;
            transform: translateY(-50%);
            background: rgba(0, 0, 0, 0.8);
            color: white;
            padding: 10px;
            border-radius: 25px;
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 5px;
            z-index: 1000;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.3);
        }
        
        .nav-btn {
            background: #4CAF50;
            color: white;
            border: none;
            width: 40px;
            height: 40px;
            border-radius: 50%;
            cursor: pointer;
            font-size: 18px;
            transition: all 0.3s ease;
        }
        
        .nav-btn:hover {
            background: #45a049;
            transform: scale(1.1);
        }
        
        .nav-btn:disabled {
            background: #666;
            cursor: not-allowed;
            transform: none;
        }
        
        #matchCounter {
            font-size: 12px;
            font-weight: bold;
            color: white;
            text-align: center;
            min-width: 40px;
        }
        
        .lang-row {
            display: flex;
            align-items: center;
            gap: 8px;
            margin: 5px 0;
        }
        
        .lang-row.lang-hidden {
            display: none !important;
        }
        
        .lang-label {
            font-weight: bold;
            min-width: 30px;
            background: rgba(255,255,255,0.2);
            padding: 2px 6px;
            border-radius: 4px;
            font-size: 0.8em;
        }
        
        .npc-context {
            background: rgba(255,255,255,0.1);
            padding: 10px;
            border-radius: 8px;
            font-size: 0.9em;
        }
        
        .dialogs {
            padding: 20px;
        }
        
        .dialog-container {
            border: 2px solid #e0e0e0;
            border-radius: 10px;
            margin: 15px 0;
            overflow: visible;
            word-wrap: break-word;
            overflow-wrap: break-word;
        }
        
        .dialog-header {
            background: #f5f5f5;
            padding: 10px 15px;
            border-bottom: 1px solid #e0e0e0;
            font-weight: bold;
            color: #333;
        }
        
        .message {
            border-left: 4px solid #4CAF50;
            margin: 10px 0;
            padding: 15px;
            background: #f9f9f9;
            border-radius: 0 8px 8px 0;
            max-width: none; /* Allow unlimited width for deep nesting */
            overflow: visible;
            word-wrap: break-word;
            overflow-wrap: break-word;
            box-sizing: border-box;
            position: relative;
            width: fit-content; /* Size to content */
            min-width: 300px; /* Minimum readable width */
        }
        
        /* Add visual indicators for very deep nesting */
        .message[data-depth="16"], .reply[data-depth="17"] { border-left-color: #ff5722; }
        .message[data-depth="17"], .reply[data-depth="18"] { border-left-color: #e91e63; }
        .message[data-depth="18"], .reply[data-depth="19"] { border-left-color: #9c27b0; }
        .message[data-depth="19"], .reply[data-depth="20"] { border-left-color: #673ab7; }
        
        /* Add depth indicator badge for very deep levels */
        
        .message-header {
            color: #2E7D32;
            margin-bottom: 10px;
            font-size: 0.9em;
        }
        
        .message-content {
            background: white;
            padding: 10px;
            border-radius: 6px;
            max-width: none; /* Allow unlimited width */
            overflow: visible;
            word-wrap: break-word;
            overflow-wrap: break-word;
        }
        
        .reply {
            border-left: 4px solid #2196F3;
            margin: 10px 0;
            padding: 15px;
            background: #e3f2fd;
            border-radius: 0 8px 8px 0;
            max-width: none; /* Allow unlimited width for deep nesting */
            overflow: visible;
            word-wrap: break-word;
            overflow-wrap: break-word;
            box-sizing: border-box;
            position: relative;
            width: fit-content; /* Size to content */
            min-width: 300px; /* Minimum readable width */
        }
        
        .reply-header {
            color: #1976D2;
            margin-bottom: 10px;
            font-size: 0.9em;
        }
        
        .reply-content {
            background: white;
            padding: 10px;
            border-radius: 6px;
            max-width: none; /* Allow unlimited width */
            overflow: visible;
            word-wrap: break-word;
            overflow-wrap: break-word;
        }
        
        .null-value {
            color: #d32f2f;
            background: #ffebee;
            padding: 2px 6px;
            border-radius: 4px;
            font-weight: bold;
            font-size: 0.9em;
        }
        
        .message-ref {
            color: #FF9800;
            font-style: italic;
            padding: 5px;
            background: #FFF3E0;
            border-radius: 4px;
            margin: 5px 0;
        }
        
        .hidden {
            display: none !important;
        }
        
        /* Dialog container styles for better deep nesting support */
        .dialogs {
            padding: 20px;
            overflow-x: auto;
            max-width: none; /* Allow unlimited width for deep nesting */
            min-width: 100%;
            /* Add scrollbar styling for better visibility */
            scrollbar-width: thin;
            scrollbar-color: #888 #f1f1f1;
        }
        
        /* Webkit scrollbar styling */
        .dialogs::-webkit-scrollbar {
            height: 8px;
        }
        
        .dialogs::-webkit-scrollbar-track {
            background: #f1f1f1;
            border-radius: 4px;
        }
        
        .dialogs::-webkit-scrollbar-thumb {
            background: #888;
            border-radius: 4px;
        }
        
        .dialogs::-webkit-scrollbar-thumb:hover {
            background: #555;
        }
        
        .dialog-container {
            border: 2px solid #e0e0e0;
            border-radius: 10px;
            margin: 15px 0;
            overflow-x: auto;
            word-wrap: break-word;
            overflow-wrap: break-word;
            min-width: 0; /* Allows container to shrink */
            max-width: none; /* Allow unlimited width */
            width: fit-content; /* Size to content */
        }
        
        .dialog-header {
            background: #f5f5f5;
            padding: 10px 15px;
            border-bottom: 1px solid #e0e0e0;
            font-weight: bold;
            color: #333;
            position: sticky;
            left: 0;
            z-index: 2;
        }
        
        /* Ensure nested content can scroll horizontally */
        .message, .reply {
            min-width: 0; /* Allows containers to shrink */
        }
        
        /* Add visual guide for very deep nesting */
        .message[data-depth^="1"]:not([data-depth="1"]),
        .reply[data-depth^="1"]:not([data-depth="1"]) {
            border-left-width: 6px;
        }
        
        .message[data-depth^="2"],
        .reply[data-depth^="2"] {
            border-left-width: 8px;
            box-shadow: 2px 0 4px rgba(0,0,0,0.1);
        }
        
        /* Depth badge styling - positioned on the left */
        .depth-badge {
            background: #ff5722;
            color: white;
            padding: 2px 6px;
            border-radius: 10px;
            font-size: 10px;
            font-weight: bold;
            margin-right: 8px;
            display: inline-block;
        }
        
        .reply .depth-badge {
            background: #e91e63;
        }
        
        /* Improved scrolling for very wide content */
        .message-content, .reply-content {
            overflow-x: auto;
            max-width: none; /* Allow unlimited width */
        }
        
        /* Depth-based indentation classes - more precise control */
        .depth-0 { margin-left: 0px; }
        .depth-1 { margin-left: 10px; }
        .depth-2 { margin-left: 20px; }
        .depth-3 { margin-left: 30px; }
        .depth-4 { margin-left: 40px; }
        .depth-5 { margin-left: 50px; }
        .depth-6 { margin-left: 60px; }
        .depth-7 { margin-left: 70px; }
        .depth-8 { margin-left: 80px; }
        .depth-9 { margin-left: 90px; }
        .depth-10 { margin-left: 100px; }
        .depth-11 { margin-left: 110px; }
        .depth-12 { margin-left: 120px; }
        .depth-13 { margin-left: 130px; }
        .depth-14 { margin-left: 140px; }
        .depth-15 { margin-left: 150px; } /* Max visual depth */
        
        /* Header improvements with help button */
        header {
            position: relative;
        }
        
        .help-btn {
            position: absolute;
            top: 20px;
            right: 20px;
            background: linear-gradient(135deg, #4CAF50, #45a049);
            color: white;
            border: none;
            width: 40px;
            height: 40px;
            border-radius: 50%;
            font-size: 18px;
            cursor: pointer;
            transition: all 0.3s ease;
            z-index: 10;
            box-shadow: 0 2px 4px rgba(0,0,0,0.2);
        }
        
        .help-btn:hover {
            background: linear-gradient(135deg, #45a049, #4CAF50);
            transform: scale(1.1);
            box-shadow: 0 4px 8px rgba(0,0,0,0.3);
        }
        
        /* Improved label alignment */
        .option-group label {
            display: flex;
            align-items: center;
            margin: 8px 0;
            cursor: pointer;
            font-size: 14px;
            line-height: 1.4;
        }
        
        .option-group label input[type="radio"],
        .option-group label input[type="checkbox"] {
            margin-right: 8px;
            margin-top: 0;
            flex-shrink: 0;
        }
        
        /* Language dropdown styling */
        .language-dropdown {
            margin-left: 8px;
            padding: 4px 8px;
            border: 1px solid #ddd;
            border-radius: 4px;
            background: white;
            font-size: 14px;
            cursor: pointer;
        }
        
        .language-dropdown:focus {
            outline: none;
            border-color: #4CAF50;
            box-shadow: 0 0 0 2px rgba(76, 175, 80, 0.2);
        }
        
        /* Language highlighting */
        .lang-row.highlighted-fr {
            border: 2px dotted #3498db;
            border-radius: 6px;
            background-color: rgba(52, 152, 219, 0.05);
            padding: 4px;
            margin: 2px 0;
        }
        
        .lang-row.highlighted-en {
            border: 2px dotted #e74c3c;
            border-radius: 6px;
            background-color: rgba(231, 76, 60, 0.05);
            padding: 4px;
            margin: 2px 0;
        }
        
        .lang-row.highlighted-es {
            border: 2px dotted #f39c12;
            border-radius: 6px;
            background-color: rgba(243, 156, 18, 0.05);
            padding: 4px;
            margin: 2px 0;
        }
        
        .lang-row.highlighted-pt {
            border: 2px dotted #9b59b6;
            border-radius: 6px;
            background-color: rgba(155, 89, 182, 0.05);
            padding: 4px;
            margin: 2px 0;
        }
        
        /* Side Menu styling */
        .side-menu {
            position: fixed;
            top: 0;
            left: -70%;
            width: 70%;
            height: 100vh;
            background: white;
            box-shadow: 4px 0 20px rgba(0,0,0,0.3);
            z-index: 1001;
            transition: left 0.4s cubic-bezier(0.25, 0.46, 0.45, 0.94);
            overflow-y: auto;
            display: flex;
            flex-direction: column;
        }
        
        .side-menu.show {
            left: 0;
        }
        
        .side-menu-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            display: flex;
            justify-content: space-between;
            align-items: center;
            position: sticky;
            top: 0;
            z-index: 1;
        }
        
        .side-menu-header h2 {
            margin: 0;
            font-size: 24px;
        }
        
        .close-menu-btn {
            background: rgba(255,255,255,0.2);
            border: none;
            color: white;
            width: 40px;
            height: 40px;
            border-radius: 50%;
            font-size: 18px;
            cursor: pointer;
            transition: all 0.3s ease;
        }
        
        .close-menu-btn:hover {
            background: rgba(255,255,255,0.3);
            transform: scale(1.1);
        }
        
        .side-menu-content {
            padding: 30px;
            flex: 1;
            overflow-y: auto;
        }
        
        .help-section {
            margin-bottom: 30px;
            background: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
            border-left: 4px solid #4CAF50;
        }
        
        .help-section h3 {
            margin: 0 0 15px 0;
            color: #333;
            font-size: 18px;
        }
        
        .help-section ul {
            margin: 0;
            padding-left: 20px;
        }
        
        .help-section li {
            margin-bottom: 10px;
            line-height: 1.6;
        }
        
        .help-section li strong {
            color: #2c3e50;
        }
        
        /* Side menu overlay */
        .side-menu-overlay {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0,0,0,0.5);
            z-index: 999;
            opacity: 0;
            visibility: hidden;
            transition: all 0.3s ease;
        }
        
        .side-menu-overlay.show {
            opacity: 1;
            visibility: visible;
        }
        
        /* Menu toggle button */
        .menu-toggle {
            position: fixed;
            top: 20px;
            right: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border: none;
            color: white;
            width: 50px;
            height: 50px;
            border-radius: 50%;
            font-size: 20px;
            cursor: pointer;
            box-shadow: 0 4px 12px rgba(0,0,0,0.2);
            transition: all 0.3s ease;
            z-index: 1000;
        }
        
        .menu-toggle:hover {
            transform: scale(1.1);
            box-shadow: 0 6px 16px rgba(0,0,0,0.3);
        }
        
        @media (max-width: 768px) {
            body {
                padding: 10px;
            }
            
            .npc-names {
                grid-template-columns: 1fr;
            }
            
            .search-options {
                flex-direction: column;
                align-items: center;
            }
            
            .side-menu {
                width: 90%;
                left: -90%;
            }
            
            .side-menu-content {
                padding: 20px;
            }
            
            .menu-toggle {
                top: 15px;
                right: 15px;
                width: 40px;
                height: 40px;
                font-size: 16px;
            }
        }
        """
    
    def _get_javascript(self) -> str:
        """Get JavaScript for search functionality"""
        return """
        document.addEventListener('DOMContentLoaded', function() {
            const searchInput = document.getElementById('searchInput');
            const searchTypeRadios = document.querySelectorAll('input[name="searchType"]');
            const showNullOnlyCheckbox = document.getElementById('showNullOnly');
            const enableImagesCheckbox = document.getElementById('enableImages');
            const enableConsolasCheckbox = document.getElementById('enableConsolas');
            const exactMatchCheckbox = document.getElementById('exactMatch');
            const ignoreDiacriticsCheckbox = document.getElementById('ignoreDiacritics');
            const useWildcardsCheckbox = document.getElementById('useWildcards');
            const showOnlyMatchDialogsCheckbox = document.getElementById('showOnlyMatchDialogs');
            const showOnlyMatchMsgReplyCheckbox = document.getElementById('showOnlyMatchMsgReply');
            const searchLangCheckboxes = document.querySelectorAll('input[name="searchLang"]');
            const displayLangCheckboxes = document.querySelectorAll('input[name="displayLang"]');
            const applyFiltersBtn = document.getElementById('applyFiltersBtn');
            const collapseAllBtn = document.getElementById('collapseAllBtn');
            const expandAllBtn = document.getElementById('expandAllBtn');
            const floatingNav = document.getElementById('floatingNav');
            const prevMatchBtn = document.getElementById('prevMatchBtn');
            const nextMatchBtn = document.getElementById('nextMatchBtn');
            const matchCounter = document.getElementById('matchCounter');
            const npcCards = document.querySelectorAll('.npc-card');
            const highlightLanguageDropdown = document.getElementById('highlightLanguage');
            const menuToggle = document.getElementById('menuToggle');
            const sideMenu = document.getElementById('sideMenu');
            const closeSideMenu = document.getElementById('closeSideMenu');
            const sideMenuOverlay = document.getElementById('sideMenuOverlay');
            
            let currentSearchState = {
                term: '',
                type: 'text',
                nullOnly: false,
                imagesEnabled: false,
                exactMatch: false,
                ignoreDiacritics: false,
                useWildcards: false,
                showOnlyMatchDialogs: false,
                showOnlyMatchMsgReply: false,
                searchLangs: ['fr', 'en', 'es', 'pt'],
                displayLangs: ['fr', 'en', 'es', 'pt']
            };
            
            let searchMatches = [];
            let currentMatchIndex = -1;
            
            // Diacritic removal function
            function removeDiacritics(str) {
                return str.normalize('NFD').replace(/[\\u0300-\\u036f]/g, '');
            }
            
            // Wildcard to regex conversion
            function wildcardToRegex(pattern) {
                const escaped = pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&');
                return escaped.replace(/\\\\\\*/g, '[^\\\\s]*');
            }
            
            // Text highlighting function
            function highlightText(text, searchTerm, ignoreDiacritics, useWildcards, exactMatch) {
                if (!searchTerm || !text) return text;
                
                let searchText = text;
                let pattern = searchTerm;
                
                if (ignoreDiacritics) {
                    searchText = removeDiacritics(text);
                    pattern = removeDiacritics(searchTerm);
                }
                
                let regex;
                if (exactMatch) {
                    regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'gi');
                } else if (useWildcards && pattern.includes('*')) {
                    regex = new RegExp(wildcardToRegex(pattern), 'gi');
                } else {
                    regex = new RegExp(pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&'), 'gi');
                }
                
                try {
                    return text.replace(regex, '<span class="search-highlight">$&</span>');
                } catch (e) {
                    return text;
                }
            }
            
            // Collapse/Expand functionality
            function toggleCollapse(element) {
                element.classList.toggle('collapsed');
                
                // When expanding a message, also expand all nested content
                if (!element.classList.contains('collapsed') && element.classList.contains('message')) {
                    // Expand all nested messages and replies within this message
                    const nestedElements = element.querySelectorAll('.collapsible');
                    nestedElements.forEach(nested => {
                        nested.classList.remove('collapsed');
                    });
                }
            }
            
            function collapseAll() {
                document.querySelectorAll('.collapsible').forEach(el => {
                    el.classList.add('collapsed');
                });
            }
            
            function expandAll() {
                document.querySelectorAll('.collapsible').forEach(el => {
                    el.classList.remove('collapsed');
                });
            }
            
            // Add click handlers for collapsible elements
            function setupCollapsibleHandlers() {
                document.querySelectorAll('.clickable-header').forEach(header => {
                    header.addEventListener('click', function(e) {
                        e.stopPropagation();
                        const collapsible = this.closest('.collapsible');
                        if (collapsible) {
                            toggleCollapse(collapsible);
                        }
                    });
                });
            }
            
            // Track changes to show the update icon
            function updateButtonState() {
                const currentState = {
                    term: searchInput.value.toLowerCase().trim(),
                    type: document.querySelector('input[name="searchType"]:checked').value,
                    nullOnly: showNullOnlyCheckbox.checked,
                    imagesEnabled: enableImagesCheckbox.checked,
                    exactMatch: exactMatchCheckbox.checked,
                    ignoreDiacritics: ignoreDiacriticsCheckbox.checked,
                    useWildcards: useWildcardsCheckbox.checked,
                    searchLangs: Array.from(searchLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value),
                    displayLangs: Array.from(displayLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value)
                };
                
                const hasChanges = JSON.stringify(currentState) !== JSON.stringify(currentSearchState);
                applyFiltersBtn.classList.toggle('changed', hasChanges);
            }
            
            // Update language display visibility
            function updateLanguageDisplay() {
                const displayLangs = Array.from(displayLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value);
                
                document.querySelectorAll('.lang-row').forEach(row => {
                    const langLabel = row.querySelector('.lang-label');
                    if (langLabel) {
                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                        row.classList.toggle('lang-hidden', !displayLangs.includes(lang));
                    }
                });
            }
            
            // Update language highlighting
            function updateLanguageHighlighting() {
                const selectedLang = highlightLanguageDropdown.value;
                
                // Remove all existing highlighting
                document.querySelectorAll('.lang-row').forEach(row => {
                    row.classList.remove('highlighted-fr', 'highlighted-en', 'highlighted-es', 'highlighted-pt');
                });
                
                // Add highlighting for selected language
                if (selectedLang) {
                    document.querySelectorAll('.lang-row').forEach(row => {
                        const langLabel = row.querySelector('.lang-label');
                        if (langLabel) {
                            const lang = langLabel.textContent.toLowerCase().replace(':', '');
                            if (lang === selectedLang) {
                                row.classList.add(`highlighted-${selectedLang}`);
                            }
                        }
                    });
                }
            }
            
            // Toggle Consolas font
            function toggleConsolasFont() {
                const body = document.body;
                if (enableConsolasCheckbox.checked) {
                    body.classList.add('consolas-font');
                } else {
                    body.classList.remove('consolas-font');
                }
            }
            
            // Side menu functionality
            function toggleSideMenu() {
                const isVisible = sideMenu.classList.contains('show');
                if (isVisible) {
                    sideMenu.classList.remove('show');
                    sideMenuOverlay.classList.remove('show');
                    document.body.style.overflow = '';
                } else {
                    sideMenu.classList.add('show');
                    sideMenuOverlay.classList.add('show');
                    document.body.style.overflow = 'hidden';
                }
            }
            
            // Image loading functionality
            function setupImageHandlers() {
                if (!enableImagesCheckbox.checked) {
                    document.querySelectorAll('.npc-image-container').forEach(container => {
                        container.style.display = 'none';
                    });
                    return;
                }
                
                npcCards.forEach(card => {
                    const imgUrls = card.dataset.imgUrls;
                    if (!imgUrls) return;
                    
                    const urls = imgUrls.split('|');
                    const imageContainer = card.querySelector('.npc-image-container');
                    const image = card.querySelector('.npc-image');
                    
                    if (imageContainer && image) {
                        card.addEventListener('mouseenter', function() {
                            if (enableImagesCheckbox.checked && urls[0]) {
                                image.src = urls[0];
                                imageContainer.style.display = 'block';
                            }
                        });
                        
                        card.addEventListener('mouseleave', function() {
                            imageContainer.style.display = 'none';
                        });
                    }
                });
            }
            
            // Search in NPC names function
            function searchInNPCNames(card, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch) {
                const npcNames = card.querySelectorAll('.npc-names .lang-row');
                for (const nameRow of npcNames) {
                    const langLabel = nameRow.querySelector('.lang-label');
                    if (langLabel) {
                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                        if (searchLangs.includes(lang)) {
                            let textContent = nameRow.textContent.toLowerCase();
                            let pattern = searchTerm;
                            
                            if (ignoreDiacritics) {
                                textContent = removeDiacritics(textContent);
                                pattern = removeDiacritics(searchTerm);
                            }
                            
                            let matches = false;
                            if (exactMatch) {
                                const regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'i');
                                matches = regex.test(textContent);
                            } else if (useWildcards && pattern.includes('*')) {
                                const regex = new RegExp(wildcardToRegex(pattern), 'i');
                                matches = regex.test(textContent);
                            } else {
                                matches = textContent.includes(pattern);
                            }
                            
                            if (matches) return true;
                        }
                    }
                }
                return false;
            }
            
            // Check if a dialog tree contains matches
            function dialogTreeContainsMatch(dialogContainer, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch) {
                if (!searchTerm) return false;
                
                // Search in all text within the dialog container
                const langRows = dialogContainer.querySelectorAll('.lang-row');
                for (const row of langRows) {
                    const langLabel = row.querySelector('.lang-label');
                    if (langLabel) {
                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                        if (searchLangs.includes(lang)) {
                            let textContent = row.textContent.toLowerCase();
                            let pattern = searchTerm;
                            
                            if (ignoreDiacritics) {
                                textContent = removeDiacritics(textContent);
                                pattern = removeDiacritics(searchTerm);
                            }
                            
                            let matches = false;
                            if (exactMatch) {
                                const regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'i');
                                matches = regex.test(textContent);
                            } else if (useWildcards && pattern.includes('*')) {
                                const regex = new RegExp(wildcardToRegex(pattern), 'i');
                                matches = regex.test(textContent);
                            } else {
                                matches = textContent.includes(pattern);
                            }
                            
                            if (matches) return true;
                        }
                    }
                }
                
                return false;
            }
            
            // Text search function with language filtering
            function searchInText(card, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch) {
                // Search in NPC names
                if (searchInNPCNames(card, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch)) {
                    return true;
                }
                
                // Search in messages and replies
                const langRows = card.querySelectorAll('.message-content .lang-row, .reply-content .lang-row');
                for (const row of langRows) {
                    const langLabel = row.querySelector('.lang-label');
                    if (langLabel) {
                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                        if (searchLangs.includes(lang)) {
                            let textContent = row.textContent.toLowerCase();
                            let pattern = searchTerm;
                            
                            if (ignoreDiacritics) {
                                textContent = removeDiacritics(textContent);
                                pattern = removeDiacritics(searchTerm);
                            }
                            
                            let matches = false;
                            if (exactMatch) {
                                const regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'i');
                                matches = regex.test(textContent);
                            } else if (useWildcards && pattern.includes('*')) {
                                const regex = new RegExp(wildcardToRegex(pattern), 'i');
                                matches = regex.test(textContent);
                            } else {
                                matches = textContent.includes(pattern);
                            }
                            
                            if (matches) return true;
                        }
                    }
                }
                
                return false;
            }
            
            // Navigation functions
            function updateNavigationButtons() {
                prevMatchBtn.disabled = currentMatchIndex <= 0;
                nextMatchBtn.disabled = currentMatchIndex >= searchMatches.length - 1;
                
                if (searchMatches.length > 0) {
                    matchCounter.textContent = `${currentMatchIndex + 1}/${searchMatches.length}`;
                    floatingNav.style.display = 'flex';
                } else {
                    matchCounter.textContent = '0/0';
                    floatingNav.style.display = 'none';
                }
            }
            
            function scrollToMatch(index) {
                if (index >= 0 && index < searchMatches.length) {
                    currentMatchIndex = index;
                    const element = searchMatches[index];
                    element.scrollIntoView({ behavior: 'smooth', block: 'center' });
                    
                    // Highlight current match
                    document.querySelectorAll('.current-match').forEach(el => {
                        el.classList.remove('current-match');
                    });
                    element.classList.add('current-match');
                    
                    updateNavigationButtons();
                }
            }
            
            function performSearch() {
                const searchTerm = searchInput.value.toLowerCase().trim();
                const searchType = document.querySelector('input[name="searchType"]:checked').value;
                const showNullOnly = showNullOnlyCheckbox.checked;
                const exactMatch = exactMatchCheckbox.checked;
                const ignoreDiacritics = ignoreDiacriticsCheckbox.checked;
                const useWildcards = useWildcardsCheckbox.checked;
                const showOnlyMatchDialogs = showOnlyMatchDialogsCheckbox.checked;
                const showOnlyMatchMsgReply = showOnlyMatchMsgReplyCheckbox.checked;
                const searchLangs = Array.from(searchLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value);
                
                // Update current state
                currentSearchState = {
                    term: searchTerm,
                    type: searchType,
                    nullOnly: showNullOnly,
                    imagesEnabled: enableImagesCheckbox.checked,
                    exactMatch: exactMatch,
                    ignoreDiacritics: ignoreDiacritics,
                    useWildcards: useWildcards,
                    showOnlyMatchDialogs: showOnlyMatchDialogs,
                    showOnlyMatchMsgReply: showOnlyMatchMsgReply,
                    searchLangs: searchLangs,
                    displayLangs: Array.from(displayLangCheckboxes).filter(cb => cb.checked).map(cb => cb.value)
                };
                
                // Clear previous highlights and matches
                document.querySelectorAll('.search-highlight').forEach(highlight => {
                    highlight.outerHTML = highlight.innerHTML;
                });
                document.querySelectorAll('.current-match').forEach(el => {
                    el.classList.remove('current-match');
                });
                searchMatches = [];
                currentMatchIndex = -1;
                
                npcCards.forEach(card => {
                    let shouldShow = false;
                    
                    if (showNullOnly) {
                        shouldShow = card.innerHTML.includes('null-value');
                    } else if (searchTerm === '') {
                        shouldShow = true;
                    } else {
                        if (searchType === 'npc-id') {
                            const npcId = card.dataset.npcId;
                            shouldShow = npcId === searchTerm;
                        } else if (searchType === 'message-id') {
                            const messageIds = Array.from(card.querySelectorAll('[data-message-id]')).map(el => el.dataset.messageId);
                            shouldShow = messageIds.includes(searchTerm);
                        } else if (searchType === 'reply-id') {
                            const replyIds = Array.from(card.querySelectorAll('[data-reply-id]')).map(el => el.dataset.replyId);
                            shouldShow = replyIds.includes(searchTerm);
                        } else if (searchType === 'npc-name') {
                            shouldShow = searchInNPCNames(card, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch);
                        } else {
                            shouldShow = searchInText(card, searchTerm, searchLangs, ignoreDiacritics, useWildcards, exactMatch);
                        }
                    }
                    
                    card.classList.toggle('hidden', !shouldShow);
                    
                    // Apply dialog tree filtering if enabled and card is shown
                    if (shouldShow && showOnlyMatchDialogs && searchTerm && 
                        (searchType === 'text' || searchType === 'npc-name')) {
                        
                        const dialogContainers = card.querySelectorAll('.dialog-container');
                        dialogContainers.forEach(dialogContainer => {
                            const hasMatch = dialogTreeContainsMatch(
                                dialogContainer, searchTerm, searchLangs, 
                                ignoreDiacritics, useWildcards, exactMatch
                            );
                            dialogContainer.classList.toggle('hidden', !hasMatch);
                        });
                    } else if (shouldShow && showOnlyMatchMsgReply && searchTerm && 
                        (searchType === 'text' || searchType === 'npc-name')) {
                        
                        // Show only matching messages and their direct replies, or matching replies and their parent messages
                        const dialogContainers = card.querySelectorAll('.dialog-container');
                        dialogContainers.forEach(dialogContainer => {
                            // Hide all messages and replies first
                            const messages = dialogContainer.querySelectorAll('.message');
                            const replies = dialogContainer.querySelectorAll('.reply');
                            
                            messages.forEach(msg => msg.classList.add('hidden'));
                            replies.forEach(reply => reply.classList.add('hidden'));
                            
                            let hasAnyMatch = false;
                            
                            // Check messages for matches and show them with direct replies
                            messages.forEach(message => {
                                const msgLangRows = message.querySelectorAll('.lang-row');
                                let messageHasMatch = false;
                                
                                for (const row of msgLangRows) {
                                    const langLabel = row.querySelector('.lang-label');
                                    if (langLabel) {
                                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                                        if (searchLangs.includes(lang)) {
                                            let textContent = row.textContent.toLowerCase();
                                            let pattern = searchTerm;
                                            
                                            if (ignoreDiacritics) {
                                                textContent = removeDiacritics(textContent);
                                                pattern = removeDiacritics(searchTerm);
                                            }
                                            
                                            let matches = false;
                                            if (exactMatch) {
                                                const regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'i');
                                                matches = regex.test(textContent);
                                            } else if (useWildcards && pattern.includes('*')) {
                                                const regex = new RegExp(wildcardToRegex(pattern), 'i');
                                                matches = regex.test(textContent);
                                            } else {
                                                matches = textContent.includes(pattern);
                                            }
                                            
                                            if (matches) {
                                                messageHasMatch = true;
                                                break;
                                            }
                                        }
                                    }
                                }
                                
                                if (messageHasMatch) {
                                    message.classList.remove('hidden');
                                    hasAnyMatch = true;
                                    
                                    // Show direct replies of this message
                                    const messageId = message.dataset.messageId;
                                    const directReplies = dialogContainer.querySelectorAll(`[data-reply-id]`);
                                    directReplies.forEach(reply => {
                                        const replyParent = reply.closest('.message');
                                        if (replyParent && replyParent.dataset.messageId === messageId) {
                                            reply.classList.remove('hidden');
                                        }
                                    });
                                }
                            });
                            
                            // Check replies for matches and show them with their parent messages
                            replies.forEach(reply => {
                                const replyLangRows = reply.querySelectorAll('.lang-row');
                                let replyHasMatch = false;
                                
                                for (const row of replyLangRows) {
                                    const langLabel = row.querySelector('.lang-label');
                                    if (langLabel) {
                                        const lang = langLabel.textContent.toLowerCase().replace(':', '');
                                        if (searchLangs.includes(lang)) {
                                            let textContent = row.textContent.toLowerCase();
                                            let pattern = searchTerm;
                                            
                                            if (ignoreDiacritics) {
                                                textContent = removeDiacritics(textContent);
                                                pattern = removeDiacritics(searchTerm);
                                            }
                                            
                                            let matches = false;
                                            if (exactMatch) {
                                                const regex = new RegExp(`\\\\b${pattern.replace(/[.*+?^${}()|[\\]\\\\]/g, '\\\\$&')}\\\\b`, 'i');
                                                matches = regex.test(textContent);
                                            } else if (useWildcards && pattern.includes('*')) {
                                                const regex = new RegExp(wildcardToRegex(pattern), 'i');
                                                matches = regex.test(textContent);
                                            } else {
                                                matches = textContent.includes(pattern);
                                            }
                                            
                                            if (matches) {
                                                replyHasMatch = true;
                                                break;
                                            }
                                        }
                                    }
                                }
                                
                                if (replyHasMatch) {
                                    reply.classList.remove('hidden');
                                    hasAnyMatch = true;
                                    
                                    // Show parent message of this reply
                                    const parentMessage = reply.closest('.message');
                                    if (parentMessage) {
                                        parentMessage.classList.remove('hidden');
                                    }
                                }
                            });
                            
                            // Hide the entire dialog container if no matches found
                            dialogContainer.classList.toggle('hidden', !hasAnyMatch);
                        });
                    } else if (shouldShow) {
                        // Show all dialog containers and their content if not filtering
                        const dialogContainers = card.querySelectorAll('.dialog-container');
                        dialogContainers.forEach(dialogContainer => {
                            dialogContainer.classList.remove('hidden');
                            // Show all messages and replies
                            const messages = dialogContainer.querySelectorAll('.message');
                            const replies = dialogContainer.querySelectorAll('.reply');
                            messages.forEach(msg => msg.classList.remove('hidden'));
                            replies.forEach(reply => reply.classList.remove('hidden'));
                        });
                    }
                    
                    if (shouldShow) {
                        searchMatches.push(card);
                        
                        // Apply highlighting if showing and text search
                        if ((searchType === 'text' || searchType === 'npc-name') && searchTerm) {
                            const langRows = card.querySelectorAll('.lang-row');
                            langRows.forEach(row => {
                                const langLabel = row.querySelector('.lang-label');
                                if (langLabel) {
                                    const lang = langLabel.textContent.toLowerCase().replace(':', '');
                                    if (searchLangs.includes(lang)) {
                                        const textNodes = Array.from(row.childNodes).filter(node => 
                                            node.nodeType === Node.TEXT_NODE || 
                                            (node.nodeType === Node.ELEMENT_NODE && !node.classList.contains('lang-label'))
                                        );
                                        
                                        textNodes.forEach(node => {
                                            if (node.nodeType === Node.TEXT_NODE) {
                                                const highlighted = highlightText(node.textContent, searchTerm, ignoreDiacritics, useWildcards, exactMatch);
                                                if (highlighted !== node.textContent) {
                                                    const span = document.createElement('span');
                                                    span.innerHTML = highlighted;
                                                    node.parentNode.replaceChild(span, node);
                                                }
                                            } else if (node.textContent) {
                                                const highlighted = highlightText(node.textContent, searchTerm, ignoreDiacritics, useWildcards, exactMatch);
                                                if (highlighted !== node.textContent) {
                                                    node.innerHTML = highlighted;
                                                }
                                            }
                                        });
                                    }
                                }
                            });
                        }
                    }
                });
                
                // Update navigation
                updateNavigationButtons();
                
                // Focus on first match if any
                if (searchMatches.length > 0) {
                    scrollToMatch(0);
                }
                
                // Update language display
                updateLanguageDisplay();
                
                // Update button state after search
                updateButtonState();
                
                // Setup image handlers after search
                setupImageHandlers();
            }
            
            // Event listeners
            searchInput.addEventListener('input', updateButtonState);
            searchTypeRadios.forEach(radio => radio.addEventListener('change', updateButtonState));
            showNullOnlyCheckbox.addEventListener('change', updateButtonState);
            enableImagesCheckbox.addEventListener('change', function() {
                updateButtonState();
                setupImageHandlers();
            });
            enableConsolasCheckbox.addEventListener('change', toggleConsolasFont);
            exactMatchCheckbox.addEventListener('change', updateButtonState);
            ignoreDiacriticsCheckbox.addEventListener('change', updateButtonState);
            useWildcardsCheckbox.addEventListener('change', updateButtonState);
            showOnlyMatchDialogsCheckbox.addEventListener('change', updateButtonState);
            showOnlyMatchMsgReplyCheckbox.addEventListener('change', updateButtonState);
            searchLangCheckboxes.forEach(checkbox => checkbox.addEventListener('change', updateButtonState));
            displayLangCheckboxes.forEach(checkbox => checkbox.addEventListener('change', function() {
                updateButtonState();
                updateLanguageDisplay();
            }));
            
            // New feature event listeners
            highlightLanguageDropdown.addEventListener('change', function() {
                updateLanguageHighlighting();
                updateButtonState();
            });
            
            menuToggle.addEventListener('click', toggleSideMenu);
            closeSideMenu.addEventListener('click', toggleSideMenu);
            
            // Close side menu when clicking on overlay
            sideMenuOverlay.addEventListener('click', function(e) {
                if (e.target === sideMenuOverlay) {
                    toggleSideMenu();
                }
            });
            
            // Button handlers
            applyFiltersBtn.addEventListener('click', performSearch);
            collapseAllBtn.addEventListener('click', collapseAll);
            expandAllBtn.addEventListener('click', expandAll);
            
            // Navigation handlers
            prevMatchBtn.addEventListener('click', () => {
                if (currentMatchIndex > 0) {
                    scrollToMatch(currentMatchIndex - 1);
                }
            });
            
            nextMatchBtn.addEventListener('click', () => {
                if (currentMatchIndex < searchMatches.length - 1) {
                    scrollToMatch(currentMatchIndex + 1);
                }
            });
            
            // Allow Enter key to trigger search
            searchInput.addEventListener('keypress', function(e) {
                if (e.key === 'Enter') {
                    performSearch();
                }
            });
            
            // Initial setup
            updateLanguageDisplay();
            updateLanguageHighlighting();
            setupImageHandlers();
            setupCollapsibleHandlers();
            updateNavigationButtons();
        });
        """

# Run Main - export HTML

In [10]:
# Create and run the NPC Dialog Mapper
def main():
    """Main function to execute the NPC dialog mapping"""
    try:
        # Initialize the mapper
        mapper = NPCDialogMapper(FOLDER)
        
        # Load all data
        mapper.load_data()
        
        # Generate HTML output
        output_filename = "npc_dialog_mapping.html"
        mapper.generate_html(output_filename)
        
        # Print some statistics
        print("\n" + "="*50)
        print("MAPPING STATISTICS")
        print("="*50)
        print(f"Total NPCs loaded: {len(mapper.npcs)}")
        print(f"NPCs with dialogs: {len([npc for npc in mapper.npcs.values() if npc.dialogs])}")
        print(f"Total messages: {len(mapper.messages)}")
        print(f"Total replies: {len(mapper.replies)}")
        print(f"Total dialogs: {len(mapper.dialogs)}")
        print(f"Total metadata entries: {len(mapper.metadata)}")
        
        # Enhanced metadata statistics
        npcs_with_metadata = sum(1 for npc in mapper.npcs.values() if npc.genders or npc.img_urls)
        npcs_with_images = sum(1 for npc in mapper.npcs.values() if npc.img_urls)
        npcs_with_gender = sum(1 for npc in mapper.npcs.values() if npc.genders)
        
        print(f"\nMETADATA STATISTICS:")
        print(f"NPCs matched with metadata: {npcs_with_metadata}")
        print(f"NPCs with gender info: {npcs_with_gender}")
        print(f"NPCs with images: {npcs_with_images}")
        
        # Gender distribution
        gender_counts = {0: 0, 1: 0, 2: 0}  # male, female, undefined
        for npc in mapper.npcs.values():
            for gender in npc.genders:
                gender_counts[gender] += 1
        
        print(f"Gender distribution: Male: {gender_counts[0]}, Female: {gender_counts[1]}, Undefined: {gender_counts[2]}")
        
        # Check for null values
        null_npc_names = sum(1 for npc in mapper.npcs.values() 
                           if not npc.name_en or not npc.name_es or not npc.name_pt)
        null_messages = sum(1 for msg in mapper.messages.values() 
                          if not msg.text_en or not msg.text_es or not msg.text_pt)
        null_replies = sum(1 for reply in mapper.replies.values() 
                         if not reply.text_en or not reply.text_es or not reply.text_pt)
        
        print(f"\nNULL VALUE STATISTICS:")
        print(f"NPCs with null translations: {null_npc_names}")
        print(f"Messages with null translations: {null_messages}")
        print(f"Replies with null translations: {null_replies}")
        
        # Top NPCs by dialog count
        npc_dialog_counts = [(npc.name_fr, len(npc.dialogs)) for npc in mapper.npcs.values() if npc.dialogs]
        npc_dialog_counts.sort(key=lambda x: x[1], reverse=True)
        
        print(f"\nTOP 10 NPCs BY DIALOG COUNT:")
        for name, count in npc_dialog_counts[:10]:
            print(f"  {name}: {count} dialogs")
        
        print(f"\nHTML file '{output_filename}' generated successfully!")
        print("You can open it in a web browser to view the mapped dialogs.")
        
    except Exception as e:
        print(f"Error: {e}")
        raise

# Run the main function
if __name__ == "__main__":
    main()

Loading NPC metadata...
Loading NPC data...
Loading messages...
Loading replies...
Loading dialogs...
Building relationships...
Matching metadata with NPCs...
Matched 602 NPCs with metadata from 1012 total NPCs
Generating HTML file: npc_dialog_mapping.html
HTML file generated successfully: npc_dialog_mapping.html

MAPPING STATISTICS
Total NPCs loaded: 1012
NPCs with dialogs: 911
Total messages: 5445
Total replies: 5092
Total dialogs: 3361
Total metadata entries: 6376

METADATA STATISTICS:
NPCs matched with metadata: 602
NPCs with gender info: 602
NPCs with images: 602
Gender distribution: Male: 462, Female: 148, Undefined: 0

NULL VALUE STATISTICS:
NPCs with null translations: 8
Messages with null translations: 39
Replies with null translations: 174

TOP 10 NPCs BY DIALOG COUNT:
  Amayiro: 182 dialogs
  Oto Mustam: 179 dialogs
  Divad Dleifrepok: 46 dialogs
  Danathor: 39 dialogs
  Assistante d'Otomaï: 35 dialogs
  Kaffra Kyper: 31 dialogs
  Elviana Tirips: 31 dialogs
  Ledrob Terceséc

# Accesory test function for mapping class algorithm

In [7]:
# Test function to examine specific NPC data
def test_npc_data(npc_id: int = None, npc_name: str = None):
    """Test function to examine specific NPC data"""
    mapper = NPCDialogMapper(FOLDER)
    mapper.load_data()
    
    if npc_id:
        if npc_id in mapper.npcs:
            npc = mapper.npcs[npc_id]
            print(f"NPC ID {npc_id}: {npc.name_fr}")
            print(f"Dialogs: {len(npc.dialogs)}")
            for dialog in npc.dialogs:
                print(f"  Dialog {dialog.dialog_id} -> Message {dialog.message_id}")
                if dialog.message_id in mapper.messages:
                    msg = mapper.messages[dialog.message_id]
                    print(f"    FR: {msg.text_fr[:100]}...")
                    print(f"    Replies: {len(msg.replies)}")
        else:
            print(f"NPC ID {npc_id} not found")
    
    elif npc_name:
        found_npcs = [npc for npc in mapper.npcs.values() 
                     if npc_name.lower() in npc.name_fr.lower()]
        for npc in found_npcs:
            print(f"Found: {npc.name_fr} (ID: {npc.npc_id}) - {len(npc.dialogs)} dialogs")

# Example usage:
# test_npc_data(npc_id=23)  # Test Charlotte
# test_npc_data(npc_name="charlotte")